<h1>Flaskを使ったデータの可視化</h1>

<p>とりあえず、データをアップロードして、plotを出力するところまで進んだので共有します。flask難しい...</p>

<h2>パッケージと初期設定</h2>

In [0]:
from flask import Flask,render_template,request,make_response, jsonify,send_file
import numpy as np
import matplotlib.pyplot as plt
import os
import io
from matplotlib.backends.backend_agg import FigureCanvasAgg

app = Flask(__name__)
UPLOAD_DIR_PATH=

**メモ**
<p>UPLOAD_DIR_PATHには、アップロードしたファイルを保存するディレクトリの絶対パスを文字列の形式で入力する。</p>

<h2>アップロード前の振る舞い</h2>

In [0]:
@app.route('/')
def upload_file():
   return render_template('hello.html', title='flask test')

**hello.html**
```
{% extends "layout.html" %}
{% block content %}
<h3>こんにちは</h3>
<p>190930No2LaCaCuo.datを入力するとplotを返します。</p>
<form action="/upload" method="post" enctype="multipart/form-data">
  <input type="file" name="uploadfiles"/>
  <input type="submit" value="submit"/>
{% endblock %}
```
**メモ**
<p>formタグのactionにはデータの送信先(URIと言うらしい)を書く。enctypeはデータの形式。</p>
<p>inputタグを変更すると、直入力の方式にしたり色々出来るみたい。</p>

[formタグについて](https://qiita.com/mikuhonda/items/f3126380d3340f3d8a2b)

**layout.html**
```
<!doctype html>
<html>
	<head>
		<title>{{ title }}</title>
	</head>
	<body>
		{% block content %}
		<!-- ここにメインコンテンツを書く -->
		{% endblock %}
	</body>
</html>
```

<h2>アップロード後の振る舞い</h2>
<p>下のように書いてみました。個別に説明を加えていきます。</p>

In [0]:
@app.route("/upload",methods=["POST"])
def after_upload():

    file = request.files['uploadfiles']
    fileName = file.filename

    file.save(os.path.join(UPLOAD_DIR_PATH,fileName))
    
    f=open(os.path.join(UPLOAD_DIR_PATH,fileName))
    dataline = f.readline()
    data1=[]
    while dataline:
        temp = dataline.strip("\n").split("\t")     #文字列に対して"\n"を除去し、"\t"で分割する処理をしている
        temp = list(map(float,temp))        #tempに保存しているリストの各要素をfloat型に変換
        data1.append(temp)        #tempをdata1に追加
        dataline=f.readline()
    f.close() 
    data1 = np.array(data1)
    fig, ax = plt.subplots()
    X=data1[:,0]
    Y=data1[:,1]
    ax.plot(X,Y)
    
    
    canvas = FigureCanvasAgg(fig)
    buf = io.BytesIO()
    canvas.print_png(buf)
    data = buf.getvalue()
    
    response = make_response(data)
    response.headers['Content-Type'] = 'image/png'
    response.headers['Content-Length'] = len(data)
    return response


if __name__ == "__main__":
    app.run(debug=True)

**メモ**

以下の2記事を参考にしました。

[アップロード等に関するまとめ記事](https://qiita.com/5zm/items/eb1ae9b349ec8a5568dc)

[matplotlibの利用(Flask)](https://qiita.com/5t111111/items/3d9efdbcc630daf0e48f)

<p>以下はコードの説明になります。</p>
<p>

```python
file = request.files['uploadfiles']
    fileName = file.filename

    file.save(os.path.join(UPLOAD_DIR_PATH,fileName))
```

</p>
<p>hello.htmlで読み込んだファイルはrequest.filesというリストの中に"uploadfiles"という名前で保存していました。とりあえず、このアップロードされてきたファイルを最初に設定したディレクトリに保存します。</p>

```python
canvas = FigureCanvasAgg(fig)
buf = io.BytesIO()
canvas.print_png(buf)
```
<p>まず、subplotsのfigをFigureCanvasAggに引き渡し、ファイルオブジェクトであるbufにpngを書き込む処理としてprint_pngを行っています。print_pngの第一引数は次の通り</p>

>filename_or_obj : str or PathLike or file-like object
    The file to write to.

[print_png](https://kite.com/python/docs/matplotlib.backends.backend_tkagg.FigureCanvas.print_png)

```python
data = buf.getvalue()
    
    response = make_response(data)
    response.headers['Content-Type'] = 'image/png'
    response.headers['Content-Length'] = len(data)
    return response
```

<p>getvalue : bufの値を全て出力します。</p>
<p>make_response : flaskの中でheaderとかいうattribute的なのがあって、それを弄りたいときはこれを使うらしい。あまりピンときてない。</p>

[make_responce](https://kite.com/python/docs/flask.make_response)

